In [12]:
import glob
import os
import subprocess
import numpy as np
import pandas as pd
import math

from tqdm import tqdm
from pathlib import Path

import re

# What's happening here?

In this notebook we will read in all the csv files and compile it to useful information. 
We will end up with comprehensive CSV files containing all usefull information. 

Add to this list:
- Responses
- RT
- Testmoment
- Ppt number
- Total time taken


Make list
Check for all instances with that name
if exists: wCam
- get that
else:
- get other



# Get Data

In [13]:
# Declare all directories:

data_dir = Path('Z:/shares/ghepmk_data/2020_Kappen_PMS')

# Create all relevant directories
date_dir = "06102021/"
data_all_dir = os.path.join(data_dir, date_dir)
data_A_dir = os.path.join(data_dir, Path('10001'))
data_B_dir = os.path.join(data_dir, Path('10002'))

# TESTMOMENT A

In [14]:
os.chdir(data_A_dir)
ACsv = glob.glob("csv/" + "*ParticipantNo*" + "*.csv", recursive = True) # get all csv's in directory
ACsvID = []
CSVy_n = []
substring = "wCAM" # This is only present in some files but is prefered

for index, item in enumerate(ACsv):
    ACsvID.append(ACsv[index].split("data_{ParticipantNo}_")[1][0:3])
    if substring in ACsv[index]:
        CSVy_n.append(True)
    else:
        CSVy_n.append(False)
        
ACsvID = set(ACsvID)

# demographics = pd.read_csv("Z:/shares/ghepmk_data/2020_Kappen_PMS/24082021/cleanData.csv", encoding = "ISO-8859-1")
demographics = pd.read_csv(data_all_dir + "cleanData.csv", encoding = "ISO-8859-1")

In [15]:
exclude = ['203', # Double moment A
           '262', # Double moment A
           '285', # Double moment A (both wCAM)
           '334', # Double moment A
           '312', # Double moment A
           '361', # Double moment A, but second time moment A also did moment B (2 days later).
           '531', # Double moment A
           '661', # Double moment A, but second time moment A also did moment B.
           '667', # Double moment A, but second time moment A also did moment B. Only 4 days between moment A and B, figure out # i might have been wrong based on video. Almost month later
           
           '105', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '106', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '107', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '111', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '118', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '119', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '123', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '124', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '125', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '127', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '778', # No demographics present. Last known participant number is 677. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '999', # No demographics present. Last known participant number is 677. So probably a testing round
           '998', # No demographics present. Last known participant number is 677. So probably a testing round
           
           '407', # Double demographics entry (seems like they first filled out with wrong mentrual duration) - figure this out
           
           '373', # Has no proper experiment order - check in R file
           
           '66_', # Weird participant number, let's see if we can figure out what the correct one would've been here -> change name of csv/webm
           '6_1', # Very hacky, because if the particpiant ends in '6' and date starts with '1' this will also get triggered. Figure out how this ends up in the csv file and try to solve
           '&à&', # Weird stuff, shouldn't be here
           '11_', # Probably the same as the others, check it out
          ]

In [16]:
index = 0
for pptnum in tqdm(ACsvID): # Loop through all file ID's
#     print("Processing: " + pptnum)
    if pptnum not in exclude: # Some participants messed up, so skip those. See earlier cell for descriptions of each
        index += 1
        # Find right file to load and read_csv in
        indices = [i for i, s in enumerate(ACsv) if pptnum in s and "wCAM" not in s] # Check if there is a file WITHOUT wCAM because that is preferred (saved later)
        if len(indices) == 1: # if one, that is perfect, read it
            data = pd.read_csv(ACsv[indices[0]])
            date = re.search(pptnum + '_(.*).csv', ACsv[indices[0]]).group(1)
        if len(indices) > 1: # if more than one, something is going wrong, figure out
            print("Too many .csv files for ppt " + pptnum + ".")
            break
        elif len(indices) == 0: # if none, probably only saved before video, so check wCAM
            indices = [i for i, s in enumerate(ACsv) if pptnum in s and "wCAM" in s] # Find instances where files exist WITH wCAM
            if len(indices) == 1: # if one, that is perfect, read it
                data = pd.read_csv(ACsv[indices[0]])
                date = re.search(pptnum + '_(.*)_wCAM.csv', ACsv[indices[0]]).group(1)
            if len(indices) > 1: # if more than one, something is going wrong, figure out
                print("Too many wCAM.csv files for ppt " + pptnum + ".")
                break
            elif len(indices) == 0: # If none, something weird is happening, as it should have appeared here then
                print("No wCAM.csv files for ppt " + pptnum + ".")
                break

        #  Now check whether they completed the experiment
        if not data['stimulus'].str.contains('beep.wav').any(): # if beep.wav is present (final thing in the experiment) they finished
            print("ppt " + pptnum + " did not finish the experiment.")
            break

        # Get data in Demographics file
    #     demoindex = [i for i, s in enumerate(demographics['participantNo'] == int(pptnum)) if s][0]
        demoindex = np.where(demographics['participantNo'] == int(pptnum))[0]
        if len(demoindex) > 1: # Too many entries for this ppt
            print("Too many demographics entries for ppt " + pptnum + ".")
            break
        elif len(demoindex) == 0:
            print("No demographics entries for ppt " + pptnum + ".")
            break
        elif len(demoindex) == 1:
            # Create indices for the trial selections
            indices = [i for i, s in enumerate(data['trial_type']) if "image-keyboard-response" in s]
            indices2 = [x+1 for x in indices]
            stimNum = len(indices) # to add constant values to dataframe
            
            # Get deduced indices for startviewing and endviewing
            indicesSTARTView= [x - 2 for x in indices2]
            indicesSTOPView= [x - 1 for x in indices2]
            
            # Make start and endbeep indices
            beepindex = [i for i, s in enumerate(data['trial_type']) if "audio-keyboard-response" in s]
            startBeep = beepindex[0]
            endBeep = beepindex[1]

            # Get correct PSS and BSRI values
            statementAB = demographics['Order'][demoindex].apply(str) == "A-B"
            statementBA = demographics['Order'][demoindex].apply(str) == "B-A"

            if statementAB.bool() == True: # If order is A-B, and this is moment A, then A is folliculair
                PSS = demographics['folliculairPSS'][demoindex].values[0]
                BSRI = demographics['folliculairBSRI'][demoindex].values[0]
            elif statementBA.bool() == True: # If order is B-A, and this is moment A, then A is luteal
                PSS = demographics['luteaalPSS'][demoindex].values[0]
                BSRI = demographics['luteaalBSRI'][demoindex].values[0]
            else:
                print("ppt " + pptnum + " does not have a valid experiment order.")
            
            df = pd.DataFrame(list(zip([pptnum] * stimNum, [demographics['Age'][demoindex].values[0]] * stimNum, [demographics['FirstMenstrual'][demoindex].values[0]] * stimNum, [demographics['MenstrualDuration'][demoindex].values[0]] * stimNum,
                                       [demographics['Exclusie'][demoindex].values[0]] * stimNum, [demographics['TrueFollicular'][demoindex].values[0]] * stimNum, [demographics['TrueLuteal'][demoindex].values[0]] * stimNum, [demographics['Isala'][demoindex].values[0]] * stimNum,
                                       [demographics['Order'][demoindex].values[0]] * stimNum, ['A'] * stimNum, [date] * stimNum, [demographics['allSymptoms'][demoindex].values[0]] * stimNum, [demographics['allDisturbance'][demoindex].values[0]] * stimNum, [demographics['allRRS'][demoindex].values[0]] * stimNum,
                                       [demographics['DASS.Total'][demoindex].values[0]] * stimNum, [demographics['DASS.Stress'][demoindex].values[0]] * stimNum, [demographics['DASS.Anxiety'][demoindex].values[0]] * stimNum, [demographics['DASS.Depresh'][demoindex].values[0]] * stimNum,
                                       [demographics['PMSScore'][demoindex].values[0]] * stimNum, [demographics['Contraception'][demoindex].values[0]] * stimNum, [PSS] * stimNum, [BSRI] * stimNum,
                                       data['stimulusShort'][indices], data['valence'][indices2], data['arousal'][indices2], data['rt'][indices2], 
                                       data['time_elapsed'][indicesSTARTView], data['time_elapsed'][indicesSTOPView], data['time_elapsed'][indices2], [data['time_elapsed'][startBeep]] * stimNum, [data['time_elapsed'][endBeep]] * stimNum, )),
                           columns =['ID', 'Age', 'FirstMenstrual', 'MenstrualDuration', 
                                     'Exclusie', 'TrueFollicular', 'TrueLuteal', 'Isala',
                                     'Order', 'Moment', 'Date', 'allSymptoms', 'allDistrubance', 'allRRS', 
                                     'DASS_Total', 'DASS_Stress', 'DASS_Anxiety', 'DASS_Depression', 
                                     'PMSScore', 'Contraception', 'PSS', 'BSRI',
                                     'Stimulus', 'Valence', 'Arousal', 'rt',
                                     'startView', 'stopView', 'stopResponse', 'startBeep', 'endBeep'])
        if index == 1:
            dataset = df
        else:
            dataset = pd.concat([dataset, df], ignore_index=True)
#             break
# dataset
datasetA = dataset

100%|████████████████████████████████████████████████████████████████████████████████| 392/392 [00:16<00:00, 24.29it/s]


# TESTMOMENT B

In [17]:
os.chdir(data_B_dir)
BCsv = glob.glob("csv/" + "*ParticipantNo*" + "*.csv", recursive = True) # get all csv's in directory
BCsvID = []
CSVy_n = []
substring = "wCAM" # This is only present in some files but is prefered

for index, item in enumerate(BCsv):
    BCsvID.append(BCsv[index].split("data_{ParticipantNo}_")[1][0:3])
    if substring in BCsv[index]:
        CSVy_n.append(True)
    else:
        CSVy_n.append(False)
        
BCsvID = set(BCsvID)

# demographics = pd.read_csv("Z:/shares/ghepmk_data/2020_Kappen_PMS/24082021/cleanData.csv", encoding = "ISO-8859-1")
demographics = pd.read_csv(data_all_dir + "cleanData.csv", encoding = "ISO-8859-1")

In [18]:
exclude = ['100', # No demographics present. First known participant number is 129.
           '106', # No demographics present. First known participant number is 129. 
           '119', # No demographics present. First known participant number is 129. 
           '125', # No demographics present. First known participant number is 129.
           '126', # No demographics present. First known participant number is 129.
           '127', # No demographics present. First known participant number is 129.
           '778', # No demographics present. Last known participant number is 677. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '789', # No demographics present. Last known participant number is 677. So probably wrong participant number. Figure out and then -> change name of csv/webm
           
           
           '350', # Double moment B, but also 1 moment A. Two moment B's 5 days apart. Weird. Figure out
           '141', # Double moment B - both wCAM
           '405', # Double moment B
           
           '407', # Double demographics entry (seems like they first filled out with wrong mentrual duration) - figure this out
           '488', # Is 4888, so not yet able to include. Retrace where the typo comes from -> check via date probably
           '459', # Same as above; is 4590
           
           '590', # Kicked out for now, because '4590' is also present (which is wrong). When 4590 is resolved, 590 can come back
           '667', # Kicked out because also '6677' present. So someone did something wrong here. Probably need to kik out 6677 (rename?) -> figure out
           
           
           '14_', # Weird stuff
           '_28', # Weird stuff
           '66_', # Weird stuff
           
           '987', # Trial run probably
           '995', # No demographics, Trial run probably
           '996', # Trial run probably
           '997', # Trial run probably
           '999', # Trial run probably
          ]

In [19]:
index = 0
for pptnum in tqdm(BCsvID): # Loop through all file ID's
#     print("Processing: " + pptnum)
    if pptnum not in exclude: # Some participants messed up, so skip those. See earlier cell for descriptions of each
        index += 1
        # Find right file to load and read_csv in
        indices = [i for i, s in enumerate(BCsv) if pptnum in s and "wCAM" not in s] # Check if there is a file WITHOUT wCAM because that is preferred (saved later)
        if len(indices) == 1: # if one, that is perfect, read it
            data = pd.read_csv(BCsv[indices[0]])
            date = re.search(pptnum + '_(.*).csv', BCsv[indices[0]]).group(1)
        if len(indices) > 1: # if more than one, something is going wrong, figure out
            print("Too many .csv files for ppt " + pptnum + ".")
            break
        elif len(indices) == 0: # if none, probably only saved before video, so check wCAM
            indices = [i for i, s in enumerate(BCsv) if pptnum in s and "wCAM" in s] # Find instances where files exist WITH wCAM
            if len(indices) == 1: # if one, that is perfect, read it
                data = pd.read_csv(BCsv[indices[0]])
                date = re.search(pptnum + '_(.*)_wCAM.csv', BCsv[indices[0]]).group(1)
            if len(indices) > 1: # if more than one, something is going wrong, figure out
                print("Too many wCAM.csv files for ppt " + pptnum + ".")
                break
            elif len(indices) == 0: # If none, something weird is happening, as it should have appeared here then
                print("No wCAM.csv files for ppt " + pptnum + ".")
                break

        #  Now check whether they completed the experiment
        if not data['stimulus'].str.contains('beep.wav').any(): # if beep.wav is present (final thing in the experiment) they finished
            print("ppt " + pptnum + " did not finish the experiment.")
            break

        # Get data in Demographics file
    #     demoindex = [i for i, s in enumerate(demographics['participantNo'] == int(pptnum)) if s][0]
        demoindex = np.where(demographics['participantNo'] == int(pptnum))[0]
        if len(demoindex) > 1: # Too many entries for this ppt
            print("Too many demographics entries for ppt " + pptnum + ".")
            break
        elif len(demoindex) == 0:
            print("No demographics entries for ppt " + pptnum + ".")
            break
        elif len(demoindex) == 1:
            # Create indices for the trial selections
            indices = [i for i, s in enumerate(data['trial_type']) if "image-keyboard-response" in s]
            indices2 = [x+1 for x in indices]
            stimNum = len(indices) # to add constant values to dataframe
            
            # Get deduced indices for startviewing and endviewing
            indicesSTARTView= [x - 2 for x in indices2]
            indicesSTOPView= [x - 1 for x in indices2]
            
            # Make start and endbeep indices
            beepindex = [i for i, s in enumerate(data['trial_type']) if "audio-keyboard-response" in s]
            startBeep = beepindex[0]
            endBeep = beepindex[1]

            # Get correct PSS and BSRI values
            statementAB = demographics['Order'][demoindex].apply(str) == "A-B"
            statementBA = demographics['Order'][demoindex].apply(str) == "B-A"

            if statementAB.bool() == True: # If order is A-B, and this is moment B, then A is luteal
                PSS = demographics['luteaalPSS'][demoindex].values[0]
                BSRI = demographics['luteaalBSRI'][demoindex].values[0]
            elif statementBA.bool() == True: # If order is B-A, and this is moment B, then A is folliculair
                PSS = demographics['folliculairPSS'][demoindex].values[0]
                BSRI = demographics['folliculairBSRI'][demoindex].values[0]
            else:
                print("ppt " + pptnum + " does not have a valid experiment order.")

            df = pd.DataFrame(list(zip([pptnum] * stimNum, [demographics['Age'][demoindex].values[0]] * stimNum, [demographics['FirstMenstrual'][demoindex].values[0]] * stimNum, [demographics['MenstrualDuration'][demoindex].values[0]] * stimNum,
                                       [demographics['Exclusie'][demoindex].values[0]] * stimNum, [demographics['TrueFollicular'][demoindex].values[0]] * stimNum, [demographics['TrueLuteal'][demoindex].values[0]] * stimNum, [demographics['Isala'][demoindex].values[0]] * stimNum,
                                       [demographics['Order'][demoindex].values[0]] * stimNum, ['B'] * stimNum, [date] * stimNum, [demographics['allSymptoms'][demoindex].values[0]] * stimNum, [demographics['allDisturbance'][demoindex].values[0]] * stimNum, [demographics['allRRS'][demoindex].values[0]] * stimNum,
                                       [demographics['DASS.Total'][demoindex].values[0]] * stimNum, [demographics['DASS.Stress'][demoindex].values[0]] * stimNum, [demographics['DASS.Anxiety'][demoindex].values[0]] * stimNum, [demographics['DASS.Depresh'][demoindex].values[0]] * stimNum,
                                       [demographics['PMSScore'][demoindex].values[0]] * stimNum, [demographics['Contraception'][demoindex].values[0]] * stimNum, [PSS] * stimNum, [BSRI] * stimNum,
                                       data['stimulusShort'][indices], data['valence'][indices2], data['arousal'][indices2], data['rt'][indices2], 
                                       data['time_elapsed'][indicesSTARTView], data['time_elapsed'][indicesSTOPView], data['time_elapsed'][indices2], [data['time_elapsed'][startBeep]] * stimNum, [data['time_elapsed'][endBeep]] * stimNum )),
                           columns =['ID', 'Age', 'FirstMenstrual', 'MenstrualDuration', 
                                     'Exclusie', 'TrueFollicular', 'TrueLuteal', 'Isala',
                                     'Order', 'Moment', 'Date', 'allSymptoms', 'allDistrubance', 'allRRS', 
                                     'DASS_Total', 'DASS_Stress', 'DASS_Anxiety', 'DASS_Depression', 
                                     'PMSScore', 'Contraception', 'PSS', 'BSRI',
                                     'Stimulus', 'Valence', 'Arousal', 'rt',
                                     'startView', 'stopView', 'stopResponse', 'startBeep', 'endBeep'])
            
        if index == 1:
            dataset = df
        else:
            dataset = pd.concat([dataset, df], ignore_index=True)
    #         break
# dataset
datasetB = dataset

100%|████████████████████████████████████████████████████████████████████████████████| 381/381 [00:16<00:00, 23.61it/s]


In [20]:
dataset = pd.concat([datasetA, datasetB], ignore_index=True)
dataset

,ID,Age,FirstMenstrual,MenstrualDuration,Exclusie,TrueFollicular,TrueLuteal,Isala,Order,Moment,...,BSRI,Stimulus,Valence,Arousal,rt,startView,stopView,stopResponse,startBeep,endBeep
0,165,21,16,28,NaN,5-12-2020,19-12-2020,NaN,B-A,A,...,305.0,2165.0,80.0,17.0,4831.810,107823,113826,118663,102071,430411
1,165,21,16,28,NaN,5-12-2020,19-12-2020,NaN,B-A,A,...,305.0,1710.0,76.0,71.0,3640.575,121667,127671,131315,102071,430411
2,165,21,16,28,NaN,5-12-2020,19-12-2020,NaN,B-A,A,...,305.0,9320.0,24.0,62.0,4747.470,134319,140322,145072,102071,430411
3,165,21,16,28,NaN,5-12-2020,19-12-2020,NaN,B-A,A,...,305.0,7480.0,66.0,24.0,5944.410,148075,154078,160026,102071,430411
4,165,21,16,28,NaN,5-12-2020,19-12-2020,NaN,B-A,A,...,305.0,7182.0,16.0,74.0,7072.090,163030,169032,176107,102071,430411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15857,535,32,13,23,NaN,7-5-2021,16-5-2021,03660,A-B,B,...,328.0,1722.0,99.0,47.0,4043.785,336236,342239,346285,83286,400540
15858,535,32,13,23,NaN,7-5-2021,16-5-2021,03660,A-B,B,...,328.0,3261.0,2.0,63.0,5605.465,349289,355296,360904,83286,400540
15859,535,32,13,23,NaN,7-5-2021,16-5-2021,03660,A-B,B,...,328.0,9594.0,84.0,78.0,3180.345,363906,369909,373091,83286,400540
15860,535,32,13,23,NaN,7-5-2021,16-5-2021,03660,A-B,B,...,328.0,9920.0,2.0,62.0,4915.485,376099,382101,387019,83286,400540


In [21]:
os.chdir(os.path.join(data_dir,date_dir))
dataset.to_csv('allPMSdata.csv')